# Python project

## Ping Ball

In [18]:
import pygame
import random
import sys
import os
# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Initialize Pygame
pygame.init()


# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 200, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

# Initialize the screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Ping Game")

# Clock for controlling the frame rate
clock = pygame.time.Clock()

# Fonts
font = pygame.font.SysFont("monospace", 30)
big_font = pygame.font.SysFont("monospace", 50)

# File for high score
high_score_file = "high_score.txt"

# Function to load the high score
def load_high_score():
    if os.path.exists(high_score_file):
        with open(high_score_file, "r") as file:
            return int(file.read())
    return 0

# Function to save the high score
def save_high_score(score):
    with open(high_score_file, "w") as file:
        file.write(str(score))

# Function to display the game over screen and handle restart
def game_over_screen(score, high_score):
    while True:
        screen.fill(BLACK)
        game_over_text = big_font.render("GAME OVER", True, RED)
        score_text = font.render(f"Score: {score}", True, WHITE)
        high_score_text = font.render(f"High Score: {high_score}", True, WHITE)
        restart_text = font.render("Press R to Restart or Q to Quit", True, WHITE)

        screen.blit(game_over_text, (SCREEN_WIDTH // 2 - game_over_text.get_width() // 2, SCREEN_HEIGHT // 2 - 100))
        screen.blit(score_text, (SCREEN_WIDTH // 2 - score_text.get_width() // 2, SCREEN_HEIGHT // 2))
        screen.blit(high_score_text, (SCREEN_WIDTH // 2 - high_score_text.get_width() // 2, SCREEN_HEIGHT // 2 + 50))
        screen.blit(restart_text, (SCREEN_WIDTH // 2 - restart_text.get_width() // 2, SCREEN_HEIGHT // 2 + 150))

        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:  # Restart the game
                    return True
                if event.key == pygame.K_q:  # Quit the game
                    pygame.quit()
                    sys.exit()

# Function to reset the game state
def reset_game():
    return {
        "ball_pos": [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2],
        "ball_speed": [random.choice([-5, 5]), 5],
        "bat_pos": [SCREEN_WIDTH // 2 - 60, SCREEN_HEIGHT - 50],
        "score": 0,
        "lives": 3,
    }

# Load the high score
high_score = load_high_score()

# Initialize game state
game_state = reset_game()

# Main game loop
while True:
    screen.fill(GREEN)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # Move the bat
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and game_state["bat_pos"][0] > 0:
        game_state["bat_pos"][0] -= 10
    if keys[pygame.K_RIGHT] and game_state["bat_pos"][0] < SCREEN_WIDTH - 120:
        game_state["bat_pos"][0] += 10

    # Move the ball
    game_state["ball_pos"][0] += game_state["ball_speed"][0]
    game_state["ball_pos"][1] += game_state["ball_speed"][1]

    # Ball collision with walls
    if game_state["ball_pos"][0] <= 10 or game_state["ball_pos"][0] >= SCREEN_WIDTH - 10:
        game_state["ball_speed"][0] = -game_state["ball_speed"][0]
    if game_state["ball_pos"][1] <= 10:
        game_state["ball_speed"][1] = -game_state["ball_speed"][1]

    # Ball collision with bat
    bat_rect = pygame.Rect(game_state["bat_pos"][0], game_state["bat_pos"][1], 120, 20)
    ball_rect = pygame.Rect(game_state["ball_pos"][0] - 10, game_state["ball_pos"][1] - 10, 20, 20)
    if bat_rect.colliderect(ball_rect):
        game_state["ball_speed"][1] = -game_state["ball_speed"][1]
        game_state["score"] += 1
        game_state["ball_speed"][0] += random.choice([-1, 1])
        game_state["ball_speed"][0] = max(min(game_state["ball_speed"][0], 10), -10)

    # Ball falls below the bat
    if game_state["ball_pos"][1] > SCREEN_HEIGHT:
        game_state["lives"] -= 1
        game_state["ball_pos"] = [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2]
        game_state["ball_speed"] = [random.choice([-5, 5]), 5]

        if game_state["lives"] == 0:
            if game_state["score"] > high_score:
                high_score = game_state["score"]
                save_high_score(high_score)

            if game_over_screen(game_state["score"], high_score):
                game_state = reset_game()
                continue

    # Draw the bat
    pygame.draw.rect(screen, BLUE, (*game_state["bat_pos"], 120, 20))

    # Draw the ball
    pygame.draw.circle(screen, YELLOW, game_state["ball_pos"], 10)

    # Display the score and lives
    score_text = font.render(f"Score: {game_state['score']}", True, WHITE)
    lives_text = font.render(f"Lives: {game_state['lives']}", True, WHITE)
    high_score_text = font.render(f"High Score: {high_score}", True, WHITE)
    screen.blit(score_text, (10, 10))
    screen.blit(lives_text, (SCREEN_WIDTH - 150, 10))
    screen.blit(high_score_text, (SCREEN_WIDTH // 2 - high_score_text.get_width() // 2, 10))

    # Update the display
    pygame.display.flip()

    # Control the frame rate
    clock.tick(60)    

SystemExit: 